## install these libraries 

In [3]:
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.mask import mask
from shapely.geometry import mapping
import numpy as np
from IPython.display import display

## Load Shapefiles
The script starts by loading two shapefiles: the raw farm boundary data and the administrative boundaries for validation.

In [4]:
# Load the first shapefile
first_shapefile_path = r"D:\raw_shape.shp"
gdf1 = gpd.read_file(first_shapefile_path)

# Load the second shapefile
second_shapefile_path = r"D:\geoBoundaries-IND-ADM5.shp"
gdf2 = gpd.read_file(second_shapefile_path)


In [26]:
gdf1 = gpd.read_file(first_shapefile_path)

# Convert GeoDataFrame to DataFrame for display
df = pd.DataFrame(gdf1.drop(columns='geometry'))  # Exclude the geometry column

# Display the DataFrame without any updates or remarks
display(df)  # This will render the DataFrame nicely in the notebook


,Farm ID,Farm area,State,District,Tehsil,Village
0,32872.0,10.00,Telangana,Jangaon,Ghanpur,Shivunipalle (CT)
1,19813.0,4.82,Telangana,Jangaon,Ghanpur,Chagal
2,26001.0,6.10,Telangana,Jangaon,Ghanpur,Chagal
3,26019.0,9.50,Telangana,Jangaon,Ghanpur,Chagal
4,57435.0,3.89,Telangana,Jangaon,Ghanpur,Thatikonda
...,...,...,...,...,...,...
385,NaN,NaN,None,None,None,None
386,NaN,NaN,None,None,None,None
387,NaN,NaN,None,None,None,None
388,NaN,NaN,None,None,None,None


## Initialize Remarks Columns
New columns are added to the GeoDataFrame to store validation remarks.

In [5]:
gdf1['Remark'] = 'correct'  # Default value for internal checks
gdf1['Boundary Remark'] = 'none'  # Default value for boundary checks
gdf1['Area Remark'] = 'none'  # Default value for area checks


## Define Utility Function
A function is defined to check if two polygons are nearly identical, which is useful for precision handling.

In [6]:
def are_nearly_identical(geom1, geom2, tolerance=1e-9):
    return geom1.equals_exact(geom2, tolerance)

## Validate Overlaps and Intersections
The script checks each farm boundary against itself for overlaps and intersections using spatial indexing for efficiency.

In [7]:
sindex1 = gdf1.sindex  # Spatial index for the first shapefile
for i, geom1 in enumerate(gdf1.geometry):
    if not geom1.is_valid:
        gdf1.loc[i, 'Remark'] = 'invalid'
        continue

    overlap_found = False
    intersect_found = False

    # Query potential overlapping geometries
    possible_matches_index = list(sindex1.intersection(geom1.bounds))
    possible_matches = gdf1.iloc[possible_matches_index]

    for j, geom2 in possible_matches.iterrows():
        if i != j:
            # Add a small buffer to handle precision issues
            buffered_geom1 = geom1.buffer(0.0001)  # Adjust the buffer size as needed
            buffered_geom2 = geom2.geometry.buffer(0.0001)

            # Check if polygons overlap
            if buffered_geom1.overlaps(buffered_geom2):
                overlap_found = True
                gdf1.loc[i, 'Remark'] = 'overlap'

            # Check if polygons intersect without being mere touches
            if buffered_geom1.intersects(buffered_geom2) and not buffered_geom1.touches(buffered_geom2):
                intersect_found = True
                if gdf1.loc[i, 'Remark'] != 'overlap':
                    gdf1.loc[i, 'Remark'] = 'intersect'

            # Check if geometries are nearly identical
            if are_nearly_identical(geom1, geom2.geometry):
                gdf1.loc[i, 'Remark'] = 'overlap'
                overlap_found = True

    # If no overlap or intersection is found, set remark to correct
    if not overlap_found and not intersect_found:
        gdf1.loc[i, 'Remark'] = 'correct'


## Checking for duplicate Farm IDs

In [8]:

farm_id_field = 'Farm ID'  # Update this to the correct field name

# Find duplicate Farm IDs
duplicates = gdf1.duplicated(subset=[farm_id_field], keep=False)

# Update remark for duplicates
gdf1.loc[duplicates, 'Remark'] = 'duplicate'


## Checking for overlaps and intersections with the second shapefile


In [9]:
# Create a spatial index for the second shapefile for efficient querying
sindex2 = gdf2.sindex

# Optional: Add a unique identifier if not present
if 'shapeName' not in gdf2.columns:
    gdf2['shapeName'] = range(len(gdf2))

# Iterate through each polygon in the first shapefile
for i, geom1 in enumerate(gdf1.geometry):
    if not geom1.is_valid:
        gdf1.loc[i, 'Remark'] = 'invalid'
        continue

    # Query potential overlapping geometries from the second shapefile
    possible_matches_index = list(sindex2.intersection(geom1.bounds))
    possible_matches = gdf2.iloc[possible_matches_index]

    for j, geom2 in possible_matches.iterrows():
        # Add a small buffer to handle precision issues
        buffered_geom1 = geom1.buffer(0.0001)  # Adjust the buffer size as needed
        buffered_geom2 = geom2.geometry.buffer(0.0001)

        # Check for overlap or intersection
        if buffered_geom1.overlaps(buffered_geom2):
            # Update the boundary remark with the shapeName from the second shapefile
            gdf1.loc[i, 'Boundary Remark'] = f"overlap with {geom2['shapeName']}"
            break  # Break if overlap is found

        elif buffered_geom1.intersects(buffered_geom2):
            # Update the boundary remark with the shapeName from the second shapefile
            gdf1.loc[i, 'Boundary Remark'] = f"intersect with {geom2['shapeName']}"
            break  # Break if intersection is found

## Reproject and Calculate Areas
The GeoDataFrame is reprojected to a specific coordinate reference system (CRS), and the area of each polygon is calculated in acres.

In [10]:
# Re-project to a projected CRS (e.g., UTM zone 43N for India)
gdf1 = gdf1.to_crs(epsg=32643)

# Calculate area in acres
gdf1['Area_ac'] = gdf1.geometry.area * 0.000247105

## Add Area Remarks
Based on the calculated areas, the script adds remarks indicating whether farms meet the minimum size requirement.

In [11]:

gdf1.loc[gdf1['Area_ac'] < 1, 'Area Remark'] = 'minimum 1 acre'
gdf1.loc[gdf1['Area_ac'] >= 1, 'Area Remark'] = 'maximum 1 acre'

In [12]:
# Rename columns to be within 10 characters
gdf1.rename(columns={'Boundary Remark': 'Bound_Rem', 'Area Remark': 'Area_Rem'}, inplace=True)

## Load LULC Raster Data
The script attempts to load a raster file containing land use land cover data and reprojects the shapefile if necessary

In [13]:

# Load the LULC raster file
lulc_raster_path = r"D:\super_LULC.tif"  # Replace with your actual path

try:
    raster = rasterio.open(lulc_raster_path)
    print("Raster loaded successfully")
    print(f"Raster CRS: {raster.crs}")
    print(f"Raster nodata value: {raster.nodata}")
except Exception as e:
    print(f"Error loading raster: {e}")
    exit()

# Reproject the shapefile to match the raster CRS if they differ
if gdf1.crs != raster.crs:
    gdf1 = gdf1.to_crs(raster.crs)
    print("Shapefile reprojected to match raster CRS")

# Add a new column for LULC remarks (ensure name length <= 10 for shapefile)
gdf1['LULC_Remark'] = 'none'  # Default value

# Define the mapping between DN values and class names
lulc_class_mapping = {
    1: "Builtup, Rural",
    2: "Agriculture, Cropland",
    3: "Builtup, Mining",
    4: "Agriculture, Cropland",
    6: "Barren",
    8: "Waterbody",
    9: "Agriculture Fallow Land",
    10: "Builtup, Urban",
    11: "Temp Waterbodies",
    12: "Forest",
    13: "Forest, Scrub Forest",
    14: "Agriculture, Cropland"
    # Add other mappings as needed
}

# Function to get the most common LULC class from a polygon mask
def get_most_common_lulc(polygon, raster):
    try:
        # Set a proper nodata value based on the raster data type
        nodata_value = raster.nodata if raster.nodata is not None else -9999  # Default nodata value
        
        # Mask the raster with the polygon geometry
        out_image, out_transform = mask(raster, [mapping(polygon)], crop=True, nodata=nodata_value)
        
        # Flatten the array and remove nodata values
        data = out_image[0].flatten()  # Ensure correct dimension is flattened
        data = data[data != nodata_value]  # Remove nodata values
        
        # Find the most common LULC value
        if len(data) > 0:
            values, counts = np.unique(data, return_counts=True)
            most_common_value = values[np.argmax(counts)]
            return int(most_common_value)  # Convert to int to ensure correct field type
        else:
            return -9999  # Use nodata value to indicate no data
    except Exception as e:
        print(f"Error processing polygon: {e}")
        return -9999  # Use nodata value to indicate error

Raster loaded successfully
Raster CRS: EPSG:3857
Raster nodata value: 4.0
Shapefile reprojected to match raster CRS


## Update Remarks with LULC Information
The most common LULC value for each farm boundary is retrieved and stored in a new remarks column.

In [14]:
for i, row in gdf1.iterrows():
    geom = row.geometry
    
    # Get the most common LULC value for the polygon
    most_common_lulc = get_most_common_lulc(geom, raster)
    
    # Map the LULC value to the corresponding class name
    lulc_class_name = lulc_class_mapping.get(most_common_lulc, 'Unknown')
    
    # Update the LULC_Remark field with the class name
    gdf1.at[i, 'LULC_Remark'] = lulc_class_name
    #print(f"Processed Farm ID {row[farm_id_field]}, LULC Remark: {lulc_class_name}")

# Ensure the 'LULC_Remark' column is of type str
gdf1['LULC_Remark'] = gdf1['LULC_Remark'].astype('str')

## Save Results
Finally, the updated GeoDataFrame is saved to a new shapefile and also exported to an Excel file.

In [16]:
output_path_shp =r"D:\farm.shp"
gdf1.to_file(output_path_shp)

print(f"Updated shapefile with remarks saved to: {output_path_shp}")

# Convert GeoDataFrame to DataFrame for Excel export
df = pd.DataFrame(gdf1.drop(columns='geometry'))

# Save to Excel
output_path_excel = r"D:\output.xlsx"
df.to_excel(output_path_excel, index=False)

print(f"Excel file saved to: {output_path_excel}")

Updated shapefile with remarks saved to: D:\pythonCourse05-04-2024\Git_Hub\farm1.shp
Excel file saved to: D:\pythonCourse05-04-2024\Git_Hub\output2.xlsx


C:\Users\SuperAdmin\AppData\Local\Temp\ipykernel_8876\262413350.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf1.to_file(output_path_shp)


In [27]:
# Convert GeoDataFrame to DataFrame for display
df = pd.DataFrame(gdf1.drop(columns='geometry'))

# Display the DataFrame
display(df)  # This will render the DataFrame nicely in the notebook


,Farm ID,Farm area,State,District,Tehsil,Village
0,32872.0,10.00,Telangana,Jangaon,Ghanpur,Shivunipalle (CT)
1,19813.0,4.82,Telangana,Jangaon,Ghanpur,Chagal
2,26001.0,6.10,Telangana,Jangaon,Ghanpur,Chagal
3,26019.0,9.50,Telangana,Jangaon,Ghanpur,Chagal
4,57435.0,3.89,Telangana,Jangaon,Ghanpur,Thatikonda
...,...,...,...,...,...,...
385,NaN,NaN,None,None,None,None
386,NaN,NaN,None,None,None,None
387,NaN,NaN,None,None,None,None
388,NaN,NaN,None,None,None,None
